In [1]:
import os
import numpy as np
import pandas as pd
import holopy as hp
from PIL import Image
from src.utils.const import *

import matlab.engine
eng = matlab.engine.start_matlab()
s = eng.genpath(os.path.join(BASEPATH, 'matlab'))
eng.addpath(s, nargout=0)

df = pd.read_csv(datapath / Path('raw/indoor_objects.csv'))
df_dict = {k: (v1, v2) for k, (v1, v2) in zip(
    df['id'], zip(df['name'], df['classification']))}

In [69]:
name = 'in_bas_13_C_20'

pluto = f'plutoScan_{name}.log'
samba = f'TraceData_{name}.csv'
eng.workspace['pluto']= os.path.join(datapath, 'new', pluto)
eng.workspace['trace']= os.path.join(datapath, 'new', samba)

eng.eval(f"[F,FI,FQ,P_X,P_Y,P_MOD,P_PHASE] = merge_acquisition(trace, pluto);",nargout=0)
eng.eval(f"[MO, PH, H, Hfill] = fast_generate_hologram(F, FI, FQ, 5, P_X, P_Y, P_MOD, P_PHASE, 2, 3);",nargout=0)

Hfill = eng.workspace['Hfill']
np_Hfill = np.asarray(Hfill, dtype = 'complex_')

# save image to file
# I8 = (((np_Hfill - np_Hfill.min()) / (np_Hfill.max() - np_Hfill.min())) * 255).astype(np.uint8)
img = Image.fromarray(np_Hfill.astype(np.uint8))
img.show()

/var/folders/yw/39qgg96x2451prd0f9qrtbzr0000gn/T/ipykernel_44415/55131469.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  img = Image.fromarray(np_Hfill.astype(np.uint8))
